# Static Stats

Produce basic stats for static GNSS testing, after removing the first 15 minutes (warm up) and last 2 minutes (shut down).

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [1]:
import os
import sys

import numpy as np

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
WARM_UP_SECS = 900
SHUTDOWN_SECS = 120

MS_TO_KNOTS = 3600 / 1852

def main():
    '''Iterate through sessions testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions', 'garden')

    errors = {}
    spikes = {}
    
    totFiles = 0
    totPoints = 0
    totDist = 0

    for root, subDirs, files in os.walk(rootDir):
        for file in sorted(files):
            ext = os.path.splitext(file)[1].lower()

            # Only interested in some file types
            if ext and ext in ['.oao']:
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        minTs = np.min(track.data['ts'])
                        maxTs = np.max(track.data['ts'])
                        
                        points = (np.argwhere((track.data['ts'] >= minTs + WARM_UP_SECS) & (track.data['ts'] <= maxTs - SHUTDOWN_SECS)))
                        
                        minSog = np.min(track.data['sog'][points]) * MS_TO_KNOTS
                        maxSog = np.max(track.data['sog'][points]) * MS_TO_KNOTS
                        meanSog = np.mean(track.data['sog'][points]) * MS_TO_KNOTS
                        medSog = np.median(track.data['sog'][points]) * MS_TO_KNOTS
                        stdSog = np.std(track.data['sog'][points]) * MS_TO_KNOTS

                        print('{}: min = {:.3f}, max = {:.3f}, mean = {:.5f}, med = {:.5f}, std = {:.5f}'.format(
                            file, minSog, maxSog, meanSog, medSog, stdSog))
                        
                    totFiles += 1

                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    print(os.linesep * 2 + 'Summary: files = {}'.format(totFiles))

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    main()
    
    print(os.linesep + 'All done!')

0630_2024-06-18-1231.oao: min = 0.000, max = 0.309, mean = 0.03731, med = 0.03499, std = 0.02120
0631_2024-06-18-1231.oao: min = 0.000, max = 0.109, mean = 0.01650, med = 0.01555, std = 0.00976
0632_2024-06-18-1231.oao: min = 0.000, max = 0.194, mean = 0.01481, med = 0.01361, std = 0.00903
0635_2024-06-18-1231.oao: min = 0.000, max = 0.134, mean = 0.02223, med = 0.01944, std = 0.01247
0660_2024-06-18-1231.oao: min = 0.000, max = 0.257, mean = 0.03184, med = 0.02916, std = 0.01787
0661_2024-06-18-1231.oao: min = 0.000, max = 0.080, mean = 0.01422, med = 0.01361, std = 0.00839
0662_2024-06-18-1232.oao: min = 0.000, max = 0.243, mean = 0.01221, med = 0.01166, std = 0.00739
0665_2024-06-18-1232.oao: min = 0.000, max = 0.156, mean = 0.02020, med = 0.01944, std = 0.01132
0801_2024-06-19-1200.oao: min = 0.000, max = 0.105, mean = 0.01026, med = 0.00972, std = 0.00661
0802_2024-06-19-1159.oao: min = 0.000, max = 0.235, mean = 0.01194, med = 0.00972, std = 0.00822
0805_2024-06-19-1159.oao: min 